In [ ]:

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
from PIL import Image
from tqdm import tqdm

# ============================
# 1) DEVICE
# ============================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

# ============================
# 2) PATHS
# ============================
DEG_DIR = "../data/train/degraded_images"
HR_DIR  = "../data/train/images"

CACHE_DIR = "cache_color"
CHECKPOINT_DIR = "checkpoints_color"
GRID_DIR = "samples_color"

os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(GRID_DIR, exist_ok=True)

# ============================
# 3) TRANSFORMS
# ============================
transform_rgb = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),  # [0,1]
])

# ============================
# 4) LUMA (pseudo grayscale)
# ============================
def rgb_to_luma(rgb01):
    r, g, b = rgb01[0], rgb01[1], rgb01[2]
    y = 0.299*r + 0.587*g + 0.114*b
    return y.unsqueeze(0)  # [1,H,W]

# ============================
# 5) DATASET + CACHE
# ============================
class ColorDataset(Dataset):
    """
    On fait simple :
    Input = Luma(degraded)  [1,H,W]
    Target = HR RGB         [3,H,W]

    Cache = accélère énormément (sinon on relit + resize + calcule à chaque epoch)
    """
    def __init__(self, deg_dir, hr_dir, cache_dir, max_items=8000):
        self.cache_dir = cache_dir

        deg_files = sorted([f for f in os.listdir(deg_dir) if f.startswith("degraded_")])
        hr_set = set(os.listdir(hr_dir))

        self.pairs = []
        for f in deg_files:
            clean = f.replace("degraded_", "")
            if clean in hr_set:
                self.pairs.append((f, clean))

        self.pairs = self.pairs[:max_items]
        print("Aligned pairs:", len(self.pairs))

        self._build_cache(deg_dir, hr_dir)

    def _build_cache(self, deg_dir, hr_dir):
        print("Building cache (only once)...")
        for i, (d, h) in enumerate(tqdm(self.pairs)):
            path = os.path.join(self.cache_dir, f"{i:06d}.pt")
            if os.path.exists(path):
                continue

            deg = Image.open(os.path.join(deg_dir, d)).convert("RGB")
            hr  = Image.open(os.path.join(hr_dir, h)).convert("RGB")

            deg = transform_rgb(deg)
            hr  = transform_rgb(hr)

            L = rgb_to_luma(deg)
            torch.save({"L": L, "target": hr}, path)

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        data = torch.load(os.path.join(self.cache_dir, f"{idx:06d}.pt"), map_location="cpu")

        # L est obligatoire
        L = data["L"]

        # target peut avoir plusieurs noms selon tes anciens scripts
        if "target" in data:
            target = data["target"]
        elif "target_rgb" in data:
            target = data["target_rgb"]
        elif "hr" in data:
            target = data["hr"]
        else:
            raise KeyError(f"Unknown keys in cache file: {data.keys()}")

        return L, target

# ============================
# 6) UNET (light)
# ============================
class DoubleConv(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_c, out_c, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_c, out_c, 3, padding=1),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.net(x)

class UNetRGB(nn.Module):
    def __init__(self, base=32):
        super().__init__()

        self.inc = DoubleConv(1, base)          # 128x128
        self.down1 = nn.Sequential(
            nn.MaxPool2d(2),                    # 64x64
            DoubleConv(base, base*2)
        )
        self.down2 = nn.Sequential(
            nn.MaxPool2d(2),                    # 32x32
            DoubleConv(base*2, base*4)
        )

        self.mid = DoubleConv(base*4, base*4)   # 32x32

        self.up1 = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=False)  # 64x64
        self.conv1 = DoubleConv(base*4 + base*2, base*2)

        self.up2 = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=False)  # 128x128
        self.conv2 = DoubleConv(base*2 + base, base)

        self.out = nn.Conv2d(base, 3, 1)

    def forward(self, x):
        x1 = self.inc(x)     # 128
        x2 = self.down1(x1)  # 64
        x3 = self.down2(x2)  # 32

        xm = self.mid(x3)

        x = self.up1(xm)              # 64
        x = torch.cat([x, x2], dim=1)
        x = self.conv1(x)

        x = self.up2(x)               # 128
        x = torch.cat([x, x1], dim=1)
        x = self.conv2(x)

        return torch.sigmoid(self.out(x))  # [0,1]

# ============================
# 7) TRAIN CONFIG
# ============================
BATCH_SIZE = 8 if DEVICE.type == "cuda" else 1
EPOCHS = 6
LR = 2e-4
MAX_ITEMS = 8000

dataset = ColorDataset(DEG_DIR, HR_DIR, CACHE_DIR, max_items=MAX_ITEMS)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

model = UNetRGB(base=32).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LR)
criterion = nn.L1Loss()

# ============================
# 8) TRAIN LOOP
# ============================
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    loop = tqdm(loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    for L, target in loop:
        L = L.to(DEVICE)
        target = target.to(DEVICE)

        pred = model(L)
        loss = criterion(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_postfix(loss=float(loss.item()))

    avg = total_loss / len(loader)
    print(f"Epoch {epoch+1} | Avg Loss: {avg:.4f}")

    torch.save(model.state_dict(), f"{CHECKPOINT_DIR}/color_epoch_{epoch+1}.pth")

    model.eval()
    with torch.no_grad():
        L, target = next(iter(loader))
        pred = model(L.to(DEVICE)).cpu()

        # grid: input(gray->rgb), pred, gt
        # On convertit L en 3 canaux pour visualiser
        L3 = L.repeat(1,3,1,1)

        grid = torch.cat([L3[:4], pred[:4], target[:4]], dim=0)
        utils.save_image(grid, f"{GRID_DIR}/epoch_{epoch+1}.png", nrow=4)

print("✅ DONE. Check:", GRID_DIR)


DEVICE: cpu
Aligned pairs: 8000
Building cache (only once)...


Epoch 1/6:   4%|▍         | 308/8000 [01:10<35:42,  3.59it/s, loss=0.13]  